In [21]:
import os

os.makedirs("data/analytics", exist_ok=True)


In [22]:
os.listdir("data")


['analytics']

In [23]:
import numpy as np
import pandas as pd


def generate_city_daily_analytical_tables():
    np.random.seed(42)

    cities = ["Shanghai", "Beijing", "Guangzhou"]
    dates = pd.date_range("2022-01-01", "2022-12-31", freq="D")

    city_base_demand = {
        "Shanghai": 1200,
        "Beijing": 1000,
        "Guangzhou": 900
    }

    city_temp_base = {
        "Shanghai": 17,
        "Beijing": 13,
        "Guangzhou": 22
    }

    records = []

    for city in cities:
        for date in dates:
            base_demand = city_base_demand[city]

            weekly_factor = 1.15 if date.weekday() >= 5 else 1.0
            annual_factor = 1 + 0.35 * np.sin(2 * np.pi * date.dayofyear / 365)

            promotion_flag = np.random.binomial(1, 0.1)
            promotion_factor = 1 + promotion_flag * 0.5

            avg_temperature = (
                city_temp_base[city]
                + 10 * np.sin(2 * np.pi * date.dayofyear / 365)
                + np.random.normal(0, 2)
            )

            temperature_factor = 1 + 0.02 * (avg_temperature - 20)
            noise = np.random.normal(0, 0.08)

            sales = (
                base_demand
                * weekly_factor
                * annual_factor
                * promotion_factor
                * temperature_factor
                * (1 + noise)
            )

            records.append({
                "city": city,
                "date": date,
                "coconut_latte_sales": max(int(sales), 0),
                "promotion_flag": promotion_flag,
                "avg_temperature": round(avg_temperature, 1)
            })

    sales_df = pd.DataFrame(records)

    MILK_PER_CUP = 0.12
    SHELF_LIFE_DAYS = 7

    milk_records = []

    for city in sales_df["city"].unique():
        city_df = sales_df[sales_df["city"] == city].sort_values("date")
        rolling_inventory = []

        for _, row in city_df.iterrows():
            daily_used = row["coconut_latte_sales"] * MILK_PER_CUP
            rolling_inventory.append(daily_used)

            expired = 0.0
            if len(rolling_inventory) > SHELF_LIFE_DAYS:
                expired = rolling_inventory.pop(0)

            milk_records.append({
                "city": city,
                "date": row["date"],
                "coconut_milk_used": round(daily_used, 2),
                "coconut_milk_expired": round(expired, 2)
            })

    milk_df = pd.DataFrame(milk_records)

    return sales_df, milk_df


In [24]:
sales_df, milk_df = generate_city_daily_analytical_tables()

sales_df.head(), milk_df.head()


(       city       date  coconut_latte_sales  promotion_flag  avg_temperature
 0  Shanghai 2022-01-01                 1279               0             14.9
 1  Shanghai 2022-01-02                 1497               0             20.5
 2  Shanghai 2022-01-03                 1085               0             16.4
 3  Shanghai 2022-01-04                 1002               0             18.2
 4  Shanghai 2022-01-05                 1074               0             19.5,
        city       date  coconut_milk_used  coconut_milk_expired
 0  Shanghai 2022-01-01             153.48                   0.0
 1  Shanghai 2022-01-02             179.64                   0.0
 2  Shanghai 2022-01-03             130.20                   0.0
 3  Shanghai 2022-01-04             120.24                   0.0
 4  Shanghai 2022-01-05             128.88                   0.0)

In [25]:
sales_df.to_csv(
    "data/analytics/city_daily_sales_for_forecast.csv",
    index=False
)

milk_df.to_csv(
    "data/analytics/city_daily_coconut_milk_usage.csv",
    index=False
)

print("Saved successfully")


Saved successfully
